# Import

In [1]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm

# att48

## Function

In [2]:
### Get points
def get_points(data='data/att48.txt'):
    with open(data) as f:
        line = f.readlines()
        list_data = []
        for index, data in enumerate(line):
            list_data.append(data.strip())

    points = []
    for i in range(len(list_data)):
        points.append((int(list_data[i].split(' ')[1]), int(list_data[i].split(' ')[2])))
    return points

### Get distance
def distance(point_i : int, point_j: int):
    return ((point_i[0] - point_j[0])**2 + (point_i[1] - point_j[1])**2)**(1/2)

def get_cost_matrix():
    cost_matrix = []
    for i in range(all_points):
        row = []
        for j in range(all_points):
            row.append(distance(get_points()[i], get_points()[j]))
        cost_matrix.append(row)
    return np.array(cost_matrix)


### DataFrame
def createDF(gen, ant, distance, paht):
    df = pd.DataFrame({'Gen' : gen, 'Ant' : ant,  'Distance' : distance, 'Path' : paht})
    return df

### Reset values
def reset_values():
    get_cost_matrix()

## Run

In [3]:
### Default values
all_points = len(get_points())
cost_matrix = get_cost_matrix() # Distance between i & j
cm = cost_matrix.copy()
n = np.array([[0 if i == j else 1 / cost_matrix[i][j] for i in range(all_points)] for j in range(all_points)]) # 1/distance
gens= 20
ants= 48
alpha= 3
beta= 3
p= .02

t0 = np.array([[0 if i==j else 1 for i in range(all_points)] for j in range(all_points)], dtype=float) # Default pheromone

list_gens_for_df = []
list_ants_for_df = []
list_total_distance_for_df = []
list_paths_for_df = []


list_pheromones = {}
list_shortest_distance_value = []
list_shortest_distance_index = []
list_shortest_path = []

for gen in range(gens):

    list_paths = {}
    list_total_distance = {}

    random_start = list(range(all_points))
    for ant in range(ants):
        print(f'Gen : {gen}, Ant : {ant}')
        list_gens_for_df.append(gen)
        list_ants_for_df.append(ant)

        allowed = [i for i in np.arange(all_points)]
        paths = []

        ### -------------------------------------------------- Start --------------------------------------------------
        start = random_start.pop(random.randrange(len(random_start))) # Random between 0 to 47
        selected = start

        paths.append(selected)
        # print(f'Start (Select) : {selected}')
        
        best_path = np.where(cost_matrix[paths[-1]] == sorted(cost_matrix[paths[-1]])[1])[0][0] # Return array, # sorted(cost_matrix[paths[-1]])[1] : ค่าที่ใกล้สุด
        distance_ij = sorted(cost_matrix[paths[-1]])[1] # index[0] : 0
        
        # print(f'Best paths : {best_path} ,Distance : {distance_ij}')
        # print(f'Paths : {paths}')
        
        ### Recheck best path
        # for best in enumerate(cost_matrix[paths[-1]]):
        #     print(best)

        allowed.remove(selected)
        for i in range(all_points):
            cost_matrix[selected][i] = 0
            cost_matrix[i][selected] = 0
        # print(f'Allowed paths: {allowed}')
        

        ### -------------------------------------------------- Denominator --------------------------------------------------
        list_t0 = []
        list_n = []
        for i in allowed:
            # print(f'Proromone [{selected}][{i}] : {t0[selected][i]} ** {alpha} = {t0[selected][i] ** alpha}')
            # print(f'n [{selected}][{i}] : {n[selected][i]} ** {beta} = {n[selected][i] ** beta}')
            list_t0.append(t0[selected][i] ** alpha)
            list_n.append(n[selected][i] ** beta)
        list_t0 = np.array(list_t0)
        list_n = np.array(list_n)
        denominator = sum(list_t0 * list_n)
        # print(f'Denominator : {denominator}')
        # print('-'*50)

        ### -------------------------------------------------- Probability --------------------------------------------------
        for city in range(len(allowed)):
            prob = {}
            for i in allowed:
                prob.update({i: (t0[paths[-1]][i]**alpha) * (n[paths[-1]][i]**beta) / denominator})
                # print(f'Denominator in prob : {denominator}')
            max_prob = sorted(prob.values())[-1]
            selected = list(prob.keys())[list(prob.values()).index(max_prob)]

            # for key, value in zip(prob.keys(), prob.values()):
            #     print(key, value)

            paths.append(selected)

            # print(f'Allowed : {allowed}')
            # print(f'Select : {selected}')

            ### Check walked path
            walked = 0
            for i in cost_matrix[paths[-1]]:
                if i == 0:
                    walked += 1
            # print(f'walked : {walked}')
            if walked != all_points:
                best_path = np.where(cost_matrix[paths[-1]] == sorted(cost_matrix[paths[-1]])[walked])[0][0]
                distance_ij = sorted(cost_matrix[paths[-1]])[walked]
                # print(f'Best paths : {best_path} ,Distance : {distance_ij}')

            # print(f'Paths : {paths}')

            ### Recheck best path
            # for best in enumerate(cost_matrix[paths[-1]]):
            #     print(best)

            ### Remove the path that has been walked
            allowed.remove(selected)
            for i in range(all_points):
                cost_matrix[selected][i] = 0
                cost_matrix[i][selected] = 0
            # print(f'Allowed paths: {allowed}')

            ### -------------------------------------------------- Denominator --------------------------------------------------
            list_t0 = []
            list_n = []
            for i in allowed:
                # print(f'Proromone [{selected}][{i}] : {t0[selected][i]} ** {alpha} = {t0[selected][i] ** alpha}')
                # print(f'n [{selected}][{i}] : {n[selected][i]} ** {beta} = {n[selected][i] ** beta}')
                list_t0.append(t0[selected][i] ** alpha)
                list_n.append(n[selected][i] ** beta)
            list_t0 = np.array(list_t0)
            list_n = np.array(list_n)
            denominator = sum(list_t0 * list_n)
            # print(f'Denominator : {denominator}')
            # print('-'*50)
        print(f'Paths : {paths}')
        list_paths.update({ant : paths})
        list_paths_for_df.append(paths)

        ### -------------------------------------------------- Total distance --------------------------------------------------
        total_distance = 0
        for i in range(len(paths)):
            if i != len(paths)-1:
                total_distance += (cm[paths[i]][paths[i+1]])

        print(f'Total distance : {total_distance}')
        list_total_distance.update({ant : total_distance})
        list_total_distance_for_df.append(total_distance)
    
        # print(f'List total distance : {list_total_distance}')
        # print(f'List total path : {list_paths}')

        ### -------------------------------------------------- Reset cost matrix --------------------------------------------------
        reset_values()

        # print(f'Pheromone before update : {sum(sum(t0))}')

    ### -------------------------------------------------- Update pheromone --------------------------------------------------
    shortest_distance_value = sorted(list_total_distance.values())[0]
    shortest_distance_index = list(list_total_distance.keys())[list(list_total_distance.values()).index(shortest_distance_value)]
    shortest_path = list_paths[shortest_distance_index]
    # lpwostp = list_paths.copy()
    # lpwostp.pop(shortest_distance_index) # List paths without shortest path
    for s in range(len(shortest_path)):
        for i in range(len(cm)):
            for j in range(len(cm)):
                try:
                    if i==shortest_path[s] and j==shortest_path[s+1]:
                        t_new = ((1-p)*t0[shortest_path[s]][shortest_path[s+1]]) + (100/shortest_distance_value)
                        t0[shortest_path[s]][shortest_path[s+1]] = t_new
                    else:
                        t_new = ((1-p)*t0[i][j])
                        t0[i][j] = t_new
                    # else:
                        # t0[i][j] = 0
                except:
                    pass

    # print(f'Pheromone after update : {sum(sum(t0))}')

    # list_pheromones.update({gen : t0})

    # print(f't0 of ant {gen} : {t0}')
    # print(f'List total distance : {list_total_distance}')
    # print(f'List total path : {list_paths}')
    print(f'Shortest distance value: {shortest_distance_value}')
    print(f'Shortest distance index: {shortest_distance_index}')
    print(f'Shortest path: {shortest_path}')
    list_shortest_distance_value.append(shortest_distance_value)
    list_shortest_distance_index.append(shortest_distance_index)
    list_shortest_path.append(shortest_path)
    print(f'-'*50)

print(f'List of shortest distance values : {list_shortest_distance_value}')


Gen : 0, Ant : 0
Paths : [44, 34, 9, 23, 41, 47, 4, 28, 1, 25, 3, 38, 31, 20, 46, 10, 22, 13, 24, 12, 11, 14, 32, 45, 43, 17, 6, 27, 35, 29, 5, 36, 18, 26, 42, 16, 19, 39, 8, 0, 7, 37, 30, 21, 15, 2, 33, 40]
Total distance : 37960.19448948187
Gen : 0, Ant : 1
Paths : [36, 18, 26, 42, 29, 5, 27, 6, 17, 35, 43, 30, 37, 8, 0, 7, 21, 15, 2, 22, 10, 11, 14, 32, 45, 39, 19, 46, 20, 12, 24, 13, 33, 40, 28, 4, 47, 38, 31, 23, 9, 41, 25, 3, 34, 44, 1, 16]
Total distance : 41836.55083825213
Gen : 0, Ant : 2
Paths : [10, 22, 13, 24, 12, 20, 46, 19, 32, 45, 14, 11, 39, 8, 0, 7, 37, 30, 43, 17, 6, 27, 35, 29, 5, 36, 18, 26, 42, 16, 2, 21, 15, 40, 33, 28, 4, 47, 38, 31, 23, 9, 41, 25, 3, 34, 44, 1]
Total distance : 36767.91960521253
Gen : 0, Ant : 3
Paths : [42, 26, 18, 36, 5, 29, 27, 6, 17, 35, 43, 30, 37, 8, 0, 7, 21, 15, 2, 22, 10, 11, 14, 32, 45, 39, 19, 46, 20, 12, 24, 13, 33, 40, 28, 4, 47, 38, 31, 23, 9, 41, 25, 3, 34, 44, 1, 16]
Total distance : 41719.430152057634
Gen : 0, Ant : 4
Paths : [3

In [4]:
list_shortest_distance_value_path = {f'{path}' : distance for path, distance in zip(list_shortest_path, list_shortest_distance_value)}
list_shortest_distance_value_path

{'[16, 26, 18, 36, 5, 29, 42, 27, 6, 17, 35, 43, 30, 37, 8, 0, 7, 21, 15, 2, 22, 10, 11, 14, 32, 45, 39, 19, 46, 20, 12, 24, 13, 33, 40, 28, 4, 47, 38, 31, 23, 9, 41, 25, 3, 34, 44, 1]': 35135.455097450664,
 '[47, 4, 28, 1, 41, 25, 3, 34, 44, 9, 23, 31, 38, 24, 13, 33, 40, 15, 21, 2, 22, 10, 11, 14, 32, 45, 39, 8, 0, 7, 37, 30, 43, 17, 6, 27, 35, 29, 5, 36, 18, 26, 42, 16, 19, 46, 20, 12]': 33574.41032751383}

## DataFrame

In [5]:
df_att48 = createDF(list_gens_for_df, list_ants_for_df, list_total_distance_for_df, list_paths_for_df)
df_att48

,Gen,Ant,Distance,Path
0,0,0,37960.194489,"[44, 34, 9, 23, 41, 47, 4, 28, 1, 25, 3, 38, 3..."
1,0,1,41836.550838,"[36, 18, 26, 42, 29, 5, 27, 6, 17, 35, 43, 30,..."
2,0,2,36767.919605,"[10, 22, 13, 24, 12, 20, 46, 19, 32, 45, 14, 1..."
3,0,3,41719.430152,"[42, 26, 18, 36, 5, 29, 27, 6, 17, 35, 43, 30,..."
4,0,4,35971.413177,"[37, 30, 43, 17, 6, 27, 35, 29, 5, 36, 18, 26,..."
...,...,...,...,...
955,19,43,33983.695191,"[34, 44, 9, 23, 31, 38, 24, 13, 33, 40, 15, 21..."
956,19,44,38310.278749,"[43, 17, 6, 27, 35, 29, 5, 36, 18, 26, 42, 16,..."
957,19,45,36099.720895,"[11, 14, 32, 45, 39, 8, 0, 7, 37, 30, 43, 17, ..."
958,19,46,35736.682441,"[21, 2, 22, 10, 11, 14, 32, 45, 39, 8, 0, 7, 3..."


## DataFrame [Shortest distace]

In [6]:
df_att48[df_att48.Distance == list_shortest_distance_value[-1]]

,Gen,Ant,Distance,Path
252,5,12,33574.410328,"[47, 4, 28, 1, 41, 25, 3, 34, 44, 9, 23, 31, 3..."
311,6,23,33574.410328,"[47, 4, 28, 1, 41, 25, 3, 34, 44, 9, 23, 31, 3..."
382,7,46,33574.410328,"[47, 4, 28, 1, 41, 25, 3, 34, 44, 9, 23, 31, 3..."
403,8,19,33574.410328,"[47, 4, 28, 1, 41, 25, 3, 34, 44, 9, 23, 31, 3..."
467,9,35,33574.410328,"[47, 4, 28, 1, 41, 25, 3, 34, 44, 9, 23, 31, 3..."
525,10,45,33574.410328,"[47, 4, 28, 1, 41, 25, 3, 34, 44, 9, 23, 31, 3..."
532,11,4,33574.410328,"[47, 4, 28, 1, 41, 25, 3, 34, 44, 9, 23, 31, 3..."
591,12,15,33574.410328,"[47, 4, 28, 1, 41, 25, 3, 34, 44, 9, 23, 31, 3..."
628,13,4,33574.410328,"[47, 4, 28, 1, 41, 25, 3, 34, 44, 9, 23, 31, 3..."
688,14,16,33574.410328,"[47, 4, 28, 1, 41, 25, 3, 34, 44, 9, 23, 31, 3..."


# berlin52

## Function

In [7]:
### Get points
def get_points(data='data/berlin52.txt'):
    with open(data) as f:
        line = f.readlines()
        list_data = []
        for index, data in enumerate(line):
            list_data.append(data.strip())

    points = []
    for i in range(len(list_data)):
        points.append((int(list_data[i].split(' ')[1]), int(list_data[i].split(' ')[2])))
    return points

### Get distance
def distance(point_i : int, point_j: int):
    return ((point_i[0] - point_j[0])**2 + (point_i[1] - point_j[1])**2)**(1/2)

def get_cost_matrix():
    cost_matrix = []
    for i in range(all_points):
        row = []
        for j in range(all_points):
            row.append(distance(get_points()[i], get_points()[j]))
        cost_matrix.append(row)
    return np.array(cost_matrix)


### DataFrame
def createDF(gen, ant, distance, paht):
    df = pd.DataFrame({'Gen' : gen, 'Ant' : ant,  'Distance' : distance, 'Path' : paht})
    return df

### Reset values
def reset_values():
    get_cost_matrix()

## Run

In [8]:
### Default values
all_points = len(get_points())
cost_matrix = get_cost_matrix() # Distance between i & j
cm = cost_matrix.copy()
n = np.array([[0 if i == j else 1 / cost_matrix[i][j] for i in range(all_points)] for j in range(all_points)]) # 1/distance
gens= 20
ants= 52
alpha= 3
beta= 3
p= .02

t0 = np.array([[0 if i==j else 1 for i in range(all_points)] for j in range(all_points)], dtype=float) # Default pheromone

list_gens_for_df = []
list_ants_for_df = []
list_total_distance_for_df = []
list_paths_for_df = []


list_pheromones = {}
list_shortest_distance_value = []
list_shortest_distance_index = []
list_shortest_path = []

for gen in range(gens):

    list_paths = {}
    list_total_distance = {}

    random_start = list(range(all_points))
    for ant in range(ants):
        print(f'Gen : {gen}, Ant : {ant}')
        list_gens_for_df.append(gen)
        list_ants_for_df.append(ant)

        allowed = [i for i in np.arange(all_points)]
        paths = []

        ### -------------------------------------------------- Start --------------------------------------------------
        start = random_start.pop(random.randrange(len(random_start))) # Random between 0 to 47
        selected = start

        paths.append(selected)
        # print(f'Start (Select) : {selected}')
        
        best_path = np.where(cost_matrix[paths[-1]] == sorted(cost_matrix[paths[-1]])[1])[0][0] # Return array, # sorted(cost_matrix[paths[-1]])[1] : ค่าที่ใกล้สุด
        distance_ij = sorted(cost_matrix[paths[-1]])[1] # index[0] : 0
        
        # print(f'Best paths : {best_path} ,Distance : {distance_ij}')
        # print(f'Paths : {paths}')
        
        ### Recheck best path
        # for best in enumerate(cost_matrix[paths[-1]]):
        #     print(best)

        allowed.remove(selected)
        for i in range(all_points):
            cost_matrix[selected][i] = 0
            cost_matrix[i][selected] = 0
        # print(f'Allowed paths: {allowed}')
        

        ### -------------------------------------------------- Denominator --------------------------------------------------
        list_t0 = []
        list_n = []
        for i in allowed:
            # print(f'Proromone [{selected}][{i}] : {t0[selected][i]} ** {alpha} = {t0[selected][i] ** alpha}')
            # print(f'n [{selected}][{i}] : {n[selected][i]} ** {beta} = {n[selected][i] ** beta}')
            list_t0.append(t0[selected][i] ** alpha)
            list_n.append(n[selected][i] ** beta)
        list_t0 = np.array(list_t0)
        list_n = np.array(list_n)
        denominator = sum(list_t0 * list_n)
        # print(f'Denominator : {denominator}')
        # print('-'*50)

        ### -------------------------------------------------- Probability --------------------------------------------------
        for city in range(len(allowed)):
            prob = {}
            for i in allowed:
                prob.update({i: (t0[paths[-1]][i]**alpha) * (n[paths[-1]][i]**beta) / denominator})
                # print(f'Denominator in prob : {denominator}')
            max_prob = sorted(prob.values())[-1]
            selected = list(prob.keys())[list(prob.values()).index(max_prob)]

            # for key, value in zip(prob.keys(), prob.values()):
            #     print(key, value)

            paths.append(selected)

            # print(f'Allowed : {allowed}')
            # print(f'Select : {selected}')

            ### Check walked path
            walked = 0
            for i in cost_matrix[paths[-1]]:
                if i == 0:
                    walked += 1
            # print(f'walked : {walked}')
            if walked != all_points:
                best_path = np.where(cost_matrix[paths[-1]] == sorted(cost_matrix[paths[-1]])[walked])[0][0]
                distance_ij = sorted(cost_matrix[paths[-1]])[walked]
                # print(f'Best paths : {best_path} ,Distance : {distance_ij}')

            # print(f'Paths : {paths}')

            ### Recheck best path
            # for best in enumerate(cost_matrix[paths[-1]]):
            #     print(best)

            ### Remove the path that has been walked
            allowed.remove(selected)
            for i in range(all_points):
                cost_matrix[selected][i] = 0
                cost_matrix[i][selected] = 0
            # print(f'Allowed paths: {allowed}')

            ### -------------------------------------------------- Denominator --------------------------------------------------
            list_t0 = []
            list_n = []
            for i in allowed:
                # print(f'Proromone [{selected}][{i}] : {t0[selected][i]} ** {alpha} = {t0[selected][i] ** alpha}')
                # print(f'n [{selected}][{i}] : {n[selected][i]} ** {beta} = {n[selected][i] ** beta}')
                list_t0.append(t0[selected][i] ** alpha)
                list_n.append(n[selected][i] ** beta)
            list_t0 = np.array(list_t0)
            list_n = np.array(list_n)
            denominator = sum(list_t0 * list_n)
            # print(f'Denominator : {denominator}')
            # print('-'*50)
        print(f'Paths : {paths}')
        list_paths.update({ant : paths})
        list_paths_for_df.append(paths)

        ### -------------------------------------------------- Total distance --------------------------------------------------
        total_distance = 0
        for i in range(len(paths)):
            if i != len(paths)-1:
                total_distance += (cm[paths[i]][paths[i+1]])

        print(f'Total distance : {total_distance}')
        list_total_distance.update({ant : total_distance})
        list_total_distance_for_df.append(total_distance)
    
        # print(f'List total distance : {list_total_distance}')
        # print(f'List total path : {list_paths}')

        ### -------------------------------------------------- Reset cost matrix --------------------------------------------------
        reset_values()

        # print(f'Pheromone before update : {sum(sum(t0))}')

    ### -------------------------------------------------- Update pheromone --------------------------------------------------
    shortest_distance_value = sorted(list_total_distance.values())[0]
    shortest_distance_index = list(list_total_distance.keys())[list(list_total_distance.values()).index(shortest_distance_value)]
    shortest_path = list_paths[shortest_distance_index]
    # lpwostp = list_paths.copy()
    # lpwostp.pop(shortest_distance_index) # List paths without shortest path
    for s in range(len(shortest_path)):
        for i in range(len(cm)):
            for j in range(len(cm)):
                try:
                    if i==shortest_path[s] and j==shortest_path[s+1]:
                        t_new = ((1-p)*t0[shortest_path[s]][shortest_path[s+1]]) + (100/shortest_distance_value)
                        t0[shortest_path[s]][shortest_path[s+1]] = t_new
                    else:
                        t_new = ((1-p)*t0[i][j])
                        t0[i][j] = t_new
                    # else:
                        # t0[i][j] = 0
                except:
                    pass

    # print(f'Pheromone after update : {sum(sum(t0))}')

    # list_pheromones.update({gen : t0})

    # print(f't0 of ant {gen} : {t0}')
    # print(f'List total distance : {list_total_distance}')
    # print(f'List total path : {list_paths}')
    print(f'Shortest distance value: {shortest_distance_value}')
    print(f'Shortest distance index: {shortest_distance_index}')
    print(f'Shortest path: {shortest_path}')
    list_shortest_distance_value.append(shortest_distance_value)
    list_shortest_distance_index.append(shortest_distance_index)
    list_shortest_path.append(shortest_path)
    print(f'-'*50)

print(f'List of shortest distance values : {list_shortest_distance_value}')


Gen : 0, Ant : 0
Paths : [29, 22, 19, 49, 15, 43, 33, 34, 35, 38, 39, 37, 36, 47, 23, 4, 14, 5, 3, 24, 45, 48, 31, 0, 21, 30, 17, 2, 18, 44, 40, 7, 9, 8, 42, 32, 50, 11, 27, 26, 25, 46, 12, 13, 51, 10, 28, 20, 16, 41, 6, 1]
Total distance : 8476.422553953138
Gen : 0, Ant : 1
Paths : [15, 49, 19, 22, 30, 17, 21, 0, 48, 31, 35, 34, 33, 38, 39, 37, 36, 47, 23, 4, 14, 5, 3, 24, 45, 43, 28, 29, 20, 16, 2, 18, 44, 40, 7, 9, 8, 42, 32, 50, 11, 27, 26, 25, 46, 12, 13, 51, 10, 41, 6, 1]
Total distance : 8775.822318800421
Gen : 0, Ant : 2
Paths : [26, 27, 25, 46, 12, 13, 51, 10, 50, 11, 24, 3, 5, 4, 14, 23, 47, 37, 39, 36, 38, 35, 34, 33, 43, 45, 15, 49, 19, 22, 30, 17, 21, 0, 48, 31, 44, 18, 40, 7, 9, 8, 42, 32, 2, 16, 20, 29, 28, 41, 6, 1]
Total distance : 8095.372888868903
Gen : 0, Ant : 3
Paths : [44, 18, 40, 7, 9, 8, 42, 14, 4, 23, 47, 37, 39, 36, 38, 35, 34, 33, 43, 45, 15, 49, 19, 22, 30, 17, 21, 0, 48, 31, 2, 16, 20, 29, 28, 24, 3, 5, 11, 27, 26, 25, 46, 12, 13, 51, 10, 50, 32, 41, 6, 1]

In [9]:
list_shortest_distance_value_path = {f'{path}' : distance for path, distance in zip(list_shortest_path, list_shortest_distance_value)}
list_shortest_distance_value_path

{'[37, 39, 36, 38, 35, 34, 33, 43, 45, 47, 23, 4, 14, 5, 3, 24, 11, 27, 26, 25, 46, 12, 13, 51, 10, 50, 32, 42, 9, 8, 7, 40, 18, 44, 31, 48, 0, 21, 30, 17, 2, 16, 20, 22, 19, 49, 15, 28, 29, 41, 6, 1]': 7310.669644787184}

## DataFrame

In [10]:
df_berlin52 = createDF(list_gens_for_df, list_ants_for_df, list_total_distance_for_df, list_paths_for_df)
df_berlin52

,Gen,Ant,Distance,Path
0,0,0,8476.422554,"[29, 22, 19, 49, 15, 43, 33, 34, 35, 38, 39, 3..."
1,0,1,8775.822319,"[15, 49, 19, 22, 30, 17, 21, 0, 48, 31, 35, 34..."
2,0,2,8095.372889,"[26, 27, 25, 46, 12, 13, 51, 10, 50, 11, 24, 3..."
3,0,3,8968.852592,"[44, 18, 40, 7, 9, 8, 42, 14, 4, 23, 47, 37, 3..."
4,0,4,8621.172721,"[23, 47, 4, 14, 5, 3, 24, 45, 43, 33, 34, 35, ..."
...,...,...,...,...
1035,19,47,8082.009559,"[49, 15, 28, 29, 41, 6, 1, 20, 22, 19, 43, 45,..."
1036,19,48,8679.015823,"[51, 10, 50, 32, 42, 9, 8, 7, 40, 18, 44, 31, ..."
1037,19,49,8382.207415,"[26, 25, 46, 12, 13, 51, 10, 50, 32, 42, 9, 8,..."
1038,19,50,8574.320618,"[13, 51, 10, 50, 32, 42, 9, 8, 7, 40, 18, 44, ..."


## DataFrame [Shortest distace]

In [11]:
df_berlin52[df_berlin52.Distance == list_shortest_distance_value[-1]]

,Gen,Ant,Distance,Path
31,0,31,7310.669645,"[37, 39, 36, 38, 35, 34, 33, 43, 45, 47, 23, 4..."
57,1,5,7310.669645,"[37, 39, 36, 38, 35, 34, 33, 43, 45, 47, 23, 4..."
147,2,43,7310.669645,"[37, 39, 36, 38, 35, 34, 33, 43, 45, 47, 23, 4..."
166,3,10,7310.669645,"[37, 39, 36, 38, 35, 34, 33, 43, 45, 47, 23, 4..."
253,4,45,7310.669645,"[37, 39, 36, 38, 35, 34, 33, 43, 45, 47, 23, 4..."
293,5,33,7310.669645,"[37, 39, 36, 38, 35, 34, 33, 43, 45, 47, 23, 4..."
358,6,46,7310.669645,"[37, 39, 36, 38, 35, 34, 33, 43, 45, 47, 23, 4..."
381,7,17,7310.669645,"[37, 39, 36, 38, 35, 34, 33, 43, 45, 47, 23, 4..."
426,8,10,7310.669645,"[37, 39, 36, 38, 35, 34, 33, 43, 45, 47, 23, 4..."
495,9,27,7310.669645,"[37, 39, 36, 38, 35, 34, 33, 43, 45, 47, 23, 4..."


# eil76

## Function

In [12]:
### Get points
def get_points(data='data/eil76.txt'):
    with open(data) as f:
        line = f.readlines()
        list_data = []
        for index, data in enumerate(line):
            list_data.append(data.strip())

    points = []
    for i in range(len(list_data)):
        points.append((int(list_data[i].split(' ')[1]), int(list_data[i].split(' ')[2])))
    return points

### Get distance
def distance(point_i : int, point_j: int):
    return ((point_i[0] - point_j[0])**2 + (point_i[1] - point_j[1])**2)**(1/2)

def get_cost_matrix():
    cost_matrix = []
    for i in range(all_points):
        row = []
        for j in range(all_points):
            row.append(distance(get_points()[i], get_points()[j]))
        cost_matrix.append(row)
    return np.array(cost_matrix)


### DataFrame
def createDF(gen, ant, distance, paht):
    df = pd.DataFrame({'Gen' : gen, 'Ant' : ant,  'Distance' : distance, 'Path' : paht})
    return df

### Reset values
def reset_values():
    get_cost_matrix()

## Run

In [13]:
### Default values
all_points = len(get_points())
cost_matrix = get_cost_matrix() # Distance between i & j
cm = cost_matrix.copy()
n = np.array([[0 if i == j else 1 / cost_matrix[i][j] for i in range(all_points)] for j in range(all_points)]) # 1/distance
gens= 20
ants= 76
alpha= 3
beta= 3
p= .02

t0 = np.array([[0 if i==j else 1 for i in range(all_points)] for j in range(all_points)], dtype=float) # Default pheromone

list_gens_for_df = []
list_ants_for_df = []
list_total_distance_for_df = []
list_paths_for_df = []


list_pheromones = {}
list_shortest_distance_value = []
list_shortest_distance_index = []
list_shortest_path = []

for gen in range(gens):

    list_paths = {}
    list_total_distance = {}

    random_start = list(range(all_points))
    for ant in range(ants):
        print(f'Gen : {gen}, Ant : {ant}')
        list_gens_for_df.append(gen)
        list_ants_for_df.append(ant)

        allowed = [i for i in np.arange(all_points)]
        paths = []

        ### -------------------------------------------------- Start --------------------------------------------------
        start = random_start.pop(random.randrange(len(random_start))) # Random between 0 to 47
        selected = start

        paths.append(selected)
        # print(f'Start (Select) : {selected}')
        
        best_path = np.where(cost_matrix[paths[-1]] == sorted(cost_matrix[paths[-1]])[1])[0][0] # Return array, # sorted(cost_matrix[paths[-1]])[1] : ค่าที่ใกล้สุด
        distance_ij = sorted(cost_matrix[paths[-1]])[1] # index[0] : 0
        
        # print(f'Best paths : {best_path} ,Distance : {distance_ij}')
        # print(f'Paths : {paths}')
        
        ### Recheck best path
        # for best in enumerate(cost_matrix[paths[-1]]):
        #     print(best)

        allowed.remove(selected)
        for i in range(all_points):
            cost_matrix[selected][i] = 0
            cost_matrix[i][selected] = 0
        # print(f'Allowed paths: {allowed}')
        

        ### -------------------------------------------------- Denominator --------------------------------------------------
        list_t0 = []
        list_n = []
        for i in allowed:
            # print(f'Proromone [{selected}][{i}] : {t0[selected][i]} ** {alpha} = {t0[selected][i] ** alpha}')
            # print(f'n [{selected}][{i}] : {n[selected][i]} ** {beta} = {n[selected][i] ** beta}')
            list_t0.append(t0[selected][i] ** alpha)
            list_n.append(n[selected][i] ** beta)
        list_t0 = np.array(list_t0)
        list_n = np.array(list_n)
        denominator = sum(list_t0 * list_n)
        # print(f'Denominator : {denominator}')
        # print('-'*50)

        ### -------------------------------------------------- Probability --------------------------------------------------
        for city in range(len(allowed)):
            prob = {}
            for i in allowed:
                prob.update({i: (t0[paths[-1]][i]**alpha) * (n[paths[-1]][i]**beta) / denominator})
                # print(f'Denominator in prob : {denominator}')
            max_prob = sorted(prob.values())[-1]
            selected = list(prob.keys())[list(prob.values()).index(max_prob)]

            # for key, value in zip(prob.keys(), prob.values()):
            #     print(key, value)

            paths.append(selected)

            # print(f'Allowed : {allowed}')
            # print(f'Select : {selected}')

            ### Check walked path
            walked = 0
            for i in cost_matrix[paths[-1]]:
                if i == 0:
                    walked += 1
            # print(f'walked : {walked}')
            if walked != all_points:
                best_path = np.where(cost_matrix[paths[-1]] == sorted(cost_matrix[paths[-1]])[walked])[0][0]
                distance_ij = sorted(cost_matrix[paths[-1]])[walked]
                # print(f'Best paths : {best_path} ,Distance : {distance_ij}')

            # print(f'Paths : {paths}')

            ### Recheck best path
            # for best in enumerate(cost_matrix[paths[-1]]):
            #     print(best)

            ### Remove the path that has been walked
            allowed.remove(selected)
            for i in range(all_points):
                cost_matrix[selected][i] = 0
                cost_matrix[i][selected] = 0
            # print(f'Allowed paths: {allowed}')

            ### -------------------------------------------------- Denominator --------------------------------------------------
            list_t0 = []
            list_n = []
            for i in allowed:
                # print(f'Proromone [{selected}][{i}] : {t0[selected][i]} ** {alpha} = {t0[selected][i] ** alpha}')
                # print(f'n [{selected}][{i}] : {n[selected][i]} ** {beta} = {n[selected][i] ** beta}')
                list_t0.append(t0[selected][i] ** alpha)
                list_n.append(n[selected][i] ** beta)
            list_t0 = np.array(list_t0)
            list_n = np.array(list_n)
            denominator = sum(list_t0 * list_n)
            # print(f'Denominator : {denominator}')
            # print('-'*50)
        print(f'Paths : {paths}')
        list_paths.update({ant : paths})
        list_paths_for_df.append(paths)

        ### -------------------------------------------------- Total distance --------------------------------------------------
        total_distance = 0
        for i in range(len(paths)):
            if i != len(paths)-1:
                total_distance += (cm[paths[i]][paths[i+1]])

        print(f'Total distance : {total_distance}')
        list_total_distance.update({ant : total_distance})
        list_total_distance_for_df.append(total_distance)
    
        # print(f'List total distance : {list_total_distance}')
        # print(f'List total path : {list_paths}')

        ### -------------------------------------------------- Reset cost matrix --------------------------------------------------
        reset_values()

        # print(f'Pheromone before update : {sum(sum(t0))}')

    ### -------------------------------------------------- Update pheromone --------------------------------------------------
    shortest_distance_value = sorted(list_total_distance.values())[0]
    shortest_distance_index = list(list_total_distance.keys())[list(list_total_distance.values()).index(shortest_distance_value)]
    shortest_path = list_paths[shortest_distance_index]
    # lpwostp = list_paths.copy()
    # lpwostp.pop(shortest_distance_index) # List paths without shortest path
    for s in range(len(shortest_path)):
        for i in range(len(cm)):
            for j in range(len(cm)):
                try:
                    if i==shortest_path[s] and j==shortest_path[s+1]:
                        t_new = ((1-p)*t0[shortest_path[s]][shortest_path[s+1]]) + (100/shortest_distance_value)
                        t0[shortest_path[s]][shortest_path[s+1]] = t_new
                    else:
                        t_new = ((1-p)*t0[i][j])
                        t0[i][j] = t_new
                    # else:
                        # t0[i][j] = 0
                except:
                    pass

    # print(f'Pheromone after update : {sum(sum(t0))}')

    # list_pheromones.update({gen : t0})

    # print(f't0 of ant {gen} : {t0}')
    # print(f'List total distance : {list_total_distance}')
    # print(f'List total path : {list_paths}')
    print(f'Shortest distance value: {shortest_distance_value}')
    print(f'Shortest distance index: {shortest_distance_index}')
    print(f'Shortest path: {shortest_path}')
    list_shortest_distance_value.append(shortest_distance_value)
    list_shortest_distance_index.append(shortest_distance_index)
    list_shortest_path.append(shortest_path)
    print(f'-'*50)

print(f'List of shortest distance values : {list_shortest_distance_value}')


Gen : 0, Ant : 0
Paths : [44, 28, 47, 46, 20, 73, 27, 61, 72, 32, 62, 15, 50, 5, 67, 74, 75, 66, 33, 45, 7, 34, 6, 52, 13, 18, 53, 12, 26, 51, 3, 29, 1, 0, 42, 40, 41, 63, 21, 60, 68, 35, 70, 59, 69, 19, 36, 4, 14, 56, 25, 11, 39, 16, 2, 43, 31, 8, 38, 71, 57, 9, 37, 64, 10, 65, 58, 30, 24, 49, 17, 23, 48, 22, 55, 54]
Total distance : 628.433218263899
Gen : 0, Ant : 1
Paths : [38, 8, 31, 43, 2, 15, 62, 32, 72, 61, 27, 73, 29, 1, 67, 74, 75, 66, 33, 45, 7, 34, 6, 52, 13, 18, 53, 12, 26, 51, 44, 28, 47, 46, 20, 35, 70, 59, 69, 19, 36, 4, 14, 56, 3, 25, 11, 39, 16, 50, 5, 0, 42, 40, 41, 63, 21, 60, 68, 22, 55, 48, 23, 17, 49, 24, 54, 30, 9, 57, 71, 37, 64, 10, 65, 58]
Total distance : 613.4683190077482
Gen : 0, Ant : 2
Paths : [69, 59, 70, 35, 46, 20, 73, 27, 61, 72, 32, 62, 15, 50, 5, 67, 74, 75, 66, 33, 45, 7, 34, 6, 52, 13, 18, 53, 12, 26, 51, 44, 28, 47, 4, 36, 19, 14, 56, 3, 29, 1, 0, 42, 40, 41, 63, 21, 60, 68, 25, 11, 39, 16, 2, 43, 31, 8, 38, 71, 57, 9, 37, 64, 10, 65, 58, 30, 24,

In [14]:
list_shortest_distance_value_path = {f'{path}' : distance for path, distance in zip(list_shortest_path, list_shortest_distance_value)}
list_shortest_distance_value_path

{'[55, 22, 62, 15, 50, 5, 67, 74, 75, 66, 33, 45, 7, 34, 6, 52, 13, 18, 53, 12, 26, 51, 44, 28, 47, 46, 20, 73, 27, 61, 72, 32, 0, 42, 40, 41, 63, 21, 60, 68, 35, 70, 59, 69, 19, 36, 4, 14, 56, 3, 29, 1, 16, 39, 11, 25, 57, 71, 38, 8, 31, 43, 2, 48, 23, 17, 49, 24, 54, 30, 9, 37, 64, 10, 65, 58]': 571.1464908173365,
 '[57, 71, 38, 8, 31, 43, 2, 48, 23, 17, 49, 24, 54, 30, 9, 37, 64, 10, 65, 58, 13, 52, 34, 6, 7, 45, 33, 51, 26, 44, 28, 47, 46, 20, 73, 27, 61, 72, 32, 0, 42, 40, 41, 63, 21, 60, 68, 35, 70, 59, 69, 19, 36, 4, 14, 56, 12, 53, 18, 66, 75, 74, 67, 5, 50, 16, 39, 11, 25, 3, 29, 1, 62, 15, 22, 55]': 565.1102262167921,
 '[2, 43, 31, 8, 38, 71, 57, 9, 37, 64, 10, 65, 58, 13, 52, 34, 6, 7, 45, 33, 51, 26, 44, 28, 47, 46, 20, 73, 27, 61, 72, 32, 0, 42, 40, 41, 63, 21, 60, 68, 35, 70, 59, 69, 19, 36, 4, 14, 56, 12, 53, 18, 66, 75, 74, 67, 5, 50, 16, 39, 11, 25, 3, 29, 1, 62, 15, 22, 55, 48, 23, 17, 49, 24, 54, 30]': 560.5726689820034}

## DataFrame

In [15]:
df_eil76 = createDF(list_gens_for_df, list_ants_for_df, list_total_distance_for_df, list_paths_for_df)
df_eil76

,Gen,Ant,Distance,Path
0,0,0,628.433218,"[44, 28, 47, 46, 20, 73, 27, 61, 72, 32, 62, 1..."
1,0,1,613.468319,"[38, 8, 31, 43, 2, 15, 62, 32, 72, 61, 27, 73,..."
2,0,2,658.420930,"[69, 59, 70, 35, 46, 20, 73, 27, 61, 72, 32, 6..."
3,0,3,579.387010,"[74, 75, 66, 33, 45, 7, 34, 6, 52, 13, 18, 53,..."
4,0,4,641.084878,"[24, 49, 17, 23, 48, 15, 62, 32, 72, 61, 27, 7..."
...,...,...,...,...
1515,19,71,592.741143,"[49, 24, 54, 30, 9, 37, 64, 10, 65, 58, 13, 52..."
1516,19,72,588.844463,"[11, 25, 3, 29, 1, 62, 15, 22, 55, 48, 23, 17,..."
1517,19,73,572.356979,"[23, 17, 49, 24, 54, 30, 9, 37, 64, 10, 65, 58..."
1518,19,74,564.220045,"[22, 55, 48, 23, 17, 49, 24, 54, 30, 9, 37, 64..."


## DataFrame [Shortest distace]

In [16]:
df_eil76[df_eil76.Distance == list_shortest_distance_value[-1]]

,Gen,Ant,Distance,Path
166,2,14,560.572669,"[2, 43, 31, 8, 38, 71, 57, 9, 37, 64, 10, 65, ..."
252,3,24,560.572669,"[2, 43, 31, 8, 38, 71, 57, 9, 37, 64, 10, 65, ..."
313,4,9,560.572669,"[2, 43, 31, 8, 38, 71, 57, 9, 37, 64, 10, 65, ..."
440,5,60,560.572669,"[2, 43, 31, 8, 38, 71, 57, 9, 37, 64, 10, 65, ..."
478,6,22,560.572669,"[2, 43, 31, 8, 38, 71, 57, 9, 37, 64, 10, 65, ..."
559,7,27,560.572669,"[2, 43, 31, 8, 38, 71, 57, 9, 37, 64, 10, 65, ..."
659,8,51,560.572669,"[2, 43, 31, 8, 38, 71, 57, 9, 37, 64, 10, 65, ..."
697,9,13,560.572669,"[2, 43, 31, 8, 38, 71, 57, 9, 37, 64, 10, 65, ..."
769,10,9,560.572669,"[2, 43, 31, 8, 38, 71, 57, 9, 37, 64, 10, 65, ..."
867,11,31,560.572669,"[2, 43, 31, 8, 38, 71, 57, 9, 37, 64, 10, 65, ..."


# kroA100

## Function

In [17]:
### Get points
def get_points(data='data/kroA100.txt'):
    with open(data) as f:
        line = f.readlines()
        list_data = []
        for index, data in enumerate(line):
            list_data.append(data.strip())

    points = []
    for i in range(len(list_data)):
        points.append((int(list_data[i].split(' ')[1]), int(list_data[i].split(' ')[2])))
    return points

### Get distance
def distance(point_i : int, point_j: int):
    return ((point_i[0] - point_j[0])**2 + (point_i[1] - point_j[1])**2)**(1/2)

def get_cost_matrix():
    cost_matrix = []
    for i in range(all_points):
        row = []
        for j in range(all_points):
            row.append(distance(get_points()[i], get_points()[j]))
        cost_matrix.append(row)
    return np.array(cost_matrix)


### DataFrame
def createDF(gen, ant, distance, paht):
    df = pd.DataFrame({'Gen' : gen, 'Ant' : ant,  'Distance' : distance, 'Path' : paht})
    return df

### Reset values
def reset_values():
    get_cost_matrix()

## Run

In [18]:
### Default values
all_points = len(get_points())
cost_matrix = get_cost_matrix() # Distance between i & j
cm = cost_matrix.copy()
n = np.array([[0 if i == j else 1 / cost_matrix[i][j] for i in range(all_points)] for j in range(all_points)]) # 1/distance
gens= 20
ants= 100
alpha= 3
beta= 3
p= .02

t0 = np.array([[0 if i==j else 1 for i in range(all_points)] for j in range(all_points)], dtype=float) # Default pheromone

list_gens_for_df = []
list_ants_for_df = []
list_total_distance_for_df = []
list_paths_for_df = []


list_pheromones = {}
list_shortest_distance_value = []
list_shortest_distance_index = []
list_shortest_path = []

for gen in range(gens):

    list_paths = {}
    list_total_distance = {}

    random_start = list(range(all_points))
    for ant in range(ants):
        print(f'Gen : {gen}, Ant : {ant}')
        list_gens_for_df.append(gen)
        list_ants_for_df.append(ant)

        allowed = [i for i in np.arange(all_points)]
        paths = []

        ### -------------------------------------------------- Start --------------------------------------------------
        start = random_start.pop(random.randrange(len(random_start))) # Random between 0 to 47
        selected = start

        paths.append(selected)
        # print(f'Start (Select) : {selected}')
        
        best_path = np.where(cost_matrix[paths[-1]] == sorted(cost_matrix[paths[-1]])[1])[0][0] # Return array, # sorted(cost_matrix[paths[-1]])[1] : ค่าที่ใกล้สุด
        distance_ij = sorted(cost_matrix[paths[-1]])[1] # index[0] : 0
        
        # print(f'Best paths : {best_path} ,Distance : {distance_ij}')
        # print(f'Paths : {paths}')
        
        ### Recheck best path
        # for best in enumerate(cost_matrix[paths[-1]]):
        #     print(best)

        allowed.remove(selected)
        for i in range(all_points):
            cost_matrix[selected][i] = 0
            cost_matrix[i][selected] = 0
        # print(f'Allowed paths: {allowed}')
        

        ### -------------------------------------------------- Denominator --------------------------------------------------
        list_t0 = []
        list_n = []
        for i in allowed:
            # print(f'Proromone [{selected}][{i}] : {t0[selected][i]} ** {alpha} = {t0[selected][i] ** alpha}')
            # print(f'n [{selected}][{i}] : {n[selected][i]} ** {beta} = {n[selected][i] ** beta}')
            list_t0.append(t0[selected][i] ** alpha)
            list_n.append(n[selected][i] ** beta)
        list_t0 = np.array(list_t0)
        list_n = np.array(list_n)
        denominator = sum(list_t0 * list_n)
        # print(f'Denominator : {denominator}')
        # print('-'*50)

        ### -------------------------------------------------- Probability --------------------------------------------------
        for city in range(len(allowed)):
            prob = {}
            for i in allowed:
                prob.update({i: (t0[paths[-1]][i]**alpha) * (n[paths[-1]][i]**beta) / denominator})
                # print(f'Denominator in prob : {denominator}')
            max_prob = sorted(prob.values())[-1]
            selected = list(prob.keys())[list(prob.values()).index(max_prob)]

            # for key, value in zip(prob.keys(), prob.values()):
            #     print(key, value)

            paths.append(selected)

            # print(f'Allowed : {allowed}')
            # print(f'Select : {selected}')

            ### Check walked path
            walked = 0
            for i in cost_matrix[paths[-1]]:
                if i == 0:
                    walked += 1
            # print(f'walked : {walked}')
            if walked != all_points:
                best_path = np.where(cost_matrix[paths[-1]] == sorted(cost_matrix[paths[-1]])[walked])[0][0]
                distance_ij = sorted(cost_matrix[paths[-1]])[walked]
                # print(f'Best paths : {best_path} ,Distance : {distance_ij}')

            # print(f'Paths : {paths}')

            ### Recheck best path
            # for best in enumerate(cost_matrix[paths[-1]]):
            #     print(best)

            ### Remove the path that has been walked
            allowed.remove(selected)
            for i in range(all_points):
                cost_matrix[selected][i] = 0
                cost_matrix[i][selected] = 0
            # print(f'Allowed paths: {allowed}')

            ### -------------------------------------------------- Denominator --------------------------------------------------
            list_t0 = []
            list_n = []
            for i in allowed:
                # print(f'Proromone [{selected}][{i}] : {t0[selected][i]} ** {alpha} = {t0[selected][i] ** alpha}')
                # print(f'n [{selected}][{i}] : {n[selected][i]} ** {beta} = {n[selected][i] ** beta}')
                list_t0.append(t0[selected][i] ** alpha)
                list_n.append(n[selected][i] ** beta)
            list_t0 = np.array(list_t0)
            list_n = np.array(list_n)
            denominator = sum(list_t0 * list_n)
            # print(f'Denominator : {denominator}')
            # print('-'*50)
        print(f'Paths : {paths}')
        list_paths.update({ant : paths})
        list_paths_for_df.append(paths)

        ### -------------------------------------------------- Total distance --------------------------------------------------
        total_distance = 0
        for i in range(len(paths)):
            if i != len(paths)-1:
                total_distance += (cm[paths[i]][paths[i+1]])

        print(f'Total distance : {total_distance}')
        list_total_distance.update({ant : total_distance})
        list_total_distance_for_df.append(total_distance)
    
        # print(f'List total distance : {list_total_distance}')
        # print(f'List total path : {list_paths}')

        ### -------------------------------------------------- Reset cost matrix --------------------------------------------------
        reset_values()

        # print(f'Pheromone before update : {sum(sum(t0))}')

    ### -------------------------------------------------- Update pheromone --------------------------------------------------
    shortest_distance_value = sorted(list_total_distance.values())[0]
    shortest_distance_index = list(list_total_distance.keys())[list(list_total_distance.values()).index(shortest_distance_value)]
    shortest_path = list_paths[shortest_distance_index]
    # lpwostp = list_paths.copy()
    # lpwostp.pop(shortest_distance_index) # List paths without shortest path
    for s in range(len(shortest_path)):
        for i in range(len(cm)):
            for j in range(len(cm)):
                try:
                    if i==shortest_path[s] and j==shortest_path[s+1]:
                        t_new = ((1-p)*t0[shortest_path[s]][shortest_path[s+1]]) + (100/shortest_distance_value)
                        t0[shortest_path[s]][shortest_path[s+1]] = t_new
                    else:
                        t_new = ((1-p)*t0[i][j])
                        t0[i][j] = t_new
                    # else:
                        # t0[i][j] = 0
                except:
                    pass

    # print(f'Pheromone after update : {sum(sum(t0))}')

    # list_pheromones.update({gen : t0})

    # print(f't0 of ant {gen} : {t0}')
    # print(f'List total distance : {list_total_distance}')
    # print(f'List total path : {list_paths}')
    print(f'Shortest distance value: {shortest_distance_value}')
    print(f'Shortest distance index: {shortest_distance_index}')
    print(f'Shortest path: {shortest_path}')
    list_shortest_distance_value.append(shortest_distance_value)
    list_shortest_distance_index.append(shortest_distance_index)
    list_shortest_path.append(shortest_path)
    print(f'-'*50)

print(f'List of shortest distance values : {list_shortest_distance_value}')


Gen : 0, Ant : 0
Paths : [40, 70, 99, 47, 13, 2, 45, 28, 33, 82, 54, 11, 26, 85, 34, 19, 56, 6, 8, 86, 50, 60, 24, 80, 68, 72, 49, 43, 1, 53, 39, 63, 67, 84, 38, 29, 95, 77, 51, 4, 36, 32, 75, 12, 94, 81, 57, 27, 92, 66, 0, 62, 5, 48, 89, 9, 83, 71, 20, 73, 58, 16, 14, 10, 31, 90, 97, 22, 44, 46, 91, 7, 41, 88, 30, 79, 55, 96, 74, 18, 52, 78, 17, 23, 37, 35, 98, 93, 21, 15, 87, 69, 65, 64, 3, 25, 76, 59, 61, 42]
Total distance : 25712.798966809405
Gen : 0, Ant : 1
Paths : [8, 6, 56, 86, 50, 60, 24, 80, 68, 72, 49, 43, 1, 53, 39, 63, 67, 84, 38, 29, 95, 77, 51, 4, 36, 32, 75, 12, 94, 81, 47, 99, 70, 40, 13, 2, 45, 28, 33, 82, 54, 11, 26, 85, 34, 19, 61, 59, 76, 22, 97, 90, 44, 31, 10, 14, 16, 73, 20, 58, 71, 9, 83, 35, 37, 23, 17, 78, 52, 87, 15, 21, 93, 69, 65, 64, 3, 96, 55, 79, 30, 88, 41, 7, 91, 0, 62, 5, 48, 89, 18, 74, 25, 98, 46, 92, 27, 66, 57, 42]
Total distance : 25854.59506476418
Gen : 0, Ant : 2
Paths : [50, 86, 8, 6, 56, 19, 11, 26, 85, 34, 61, 59, 76, 22, 97, 90, 44, 31, 1

In [19]:
list_shortest_distance_value_path = {f'{path}' : distance for path, distance in zip(list_shortest_path, list_shortest_distance_value)}
list_shortest_distance_value_path

{'[3, 64, 65, 25, 69, 21, 15, 87, 93, 17, 23, 37, 35, 98, 83, 9, 71, 20, 73, 58, 16, 14, 10, 31, 90, 97, 22, 44, 46, 62, 5, 48, 89, 78, 52, 18, 74, 96, 55, 79, 30, 88, 41, 7, 91, 0, 92, 27, 66, 57, 60, 24, 80, 68, 72, 49, 43, 1, 53, 39, 63, 67, 84, 38, 29, 95, 77, 51, 4, 36, 32, 75, 12, 94, 81, 47, 99, 70, 40, 13, 2, 45, 28, 33, 82, 54, 11, 26, 85, 34, 19, 56, 6, 8, 86, 50, 76, 59, 61, 42]': 23137.514597468155,
 '[19, 56, 6, 8, 86, 50, 76, 59, 61, 85, 34, 26, 11, 54, 82, 33, 28, 45, 2, 42, 13, 70, 40, 99, 47, 51, 77, 95, 4, 36, 32, 75, 12, 94, 81, 49, 43, 1, 53, 39, 63, 68, 72, 67, 84, 38, 29, 80, 24, 60, 57, 27, 92, 66, 0, 62, 5, 48, 89, 78, 52, 18, 74, 96, 55, 79, 30, 88, 41, 7, 91, 46, 31, 90, 97, 22, 44, 10, 14, 16, 73, 58, 20, 71, 9, 83, 35, 98, 37, 23, 17, 93, 21, 15, 87, 69, 65, 64, 3, 25]': 22364.28417536669}

## DataFrame

In [20]:
df_kroA100 = createDF(list_gens_for_df, list_ants_for_df, list_total_distance_for_df, list_paths_for_df)
df_kroA100

,Gen,Ant,Distance,Path
0,0,0,25712.798967,"[40, 70, 99, 47, 13, 2, 45, 28, 33, 82, 54, 11..."
1,0,1,25854.595065,"[8, 6, 56, 86, 50, 60, 24, 80, 68, 72, 49, 43,..."
2,0,2,24783.114036,"[50, 86, 8, 6, 56, 19, 11, 26, 85, 34, 61, 59,..."
3,0,3,26425.623924,"[41, 88, 30, 79, 55, 96, 74, 18, 52, 78, 17, 2..."
4,0,4,24386.232105,"[91, 7, 41, 88, 30, 79, 55, 96, 74, 18, 52, 78..."
...,...,...,...,...
1995,19,95,25220.332999,"[88, 41, 7, 91, 46, 31, 90, 97, 22, 44, 10, 14..."
1996,19,96,25323.693961,"[70, 40, 99, 47, 51, 77, 95, 4, 36, 32, 75, 12..."
1997,19,97,24505.693993,"[26, 11, 54, 82, 33, 28, 45, 2, 42, 13, 70, 40..."
1998,19,98,24731.132283,"[55, 79, 30, 88, 41, 7, 91, 46, 31, 90, 97, 22..."


## DataFrame [Shortest distace]

In [21]:
df_kroA100[df_kroA100.Distance == list_shortest_distance_value[-1]]

,Gen,Ant,Distance,Path
391,3,91,22364.284175,"[19, 56, 6, 8, 86, 50, 76, 59, 61, 85, 34, 26,..."
459,4,59,22364.284175,"[19, 56, 6, 8, 86, 50, 76, 59, 61, 85, 34, 26,..."
511,5,11,22364.284175,"[19, 56, 6, 8, 86, 50, 76, 59, 61, 85, 34, 26,..."
692,6,92,22364.284175,"[19, 56, 6, 8, 86, 50, 76, 59, 61, 85, 34, 26,..."
793,7,93,22364.284175,"[19, 56, 6, 8, 86, 50, 76, 59, 61, 85, 34, 26,..."
862,8,62,22364.284175,"[19, 56, 6, 8, 86, 50, 76, 59, 61, 85, 34, 26,..."
915,9,15,22364.284175,"[19, 56, 6, 8, 86, 50, 76, 59, 61, 85, 34, 26,..."
1018,10,18,22364.284175,"[19, 56, 6, 8, 86, 50, 76, 59, 61, 85, 34, 26,..."
1117,11,17,22364.284175,"[19, 56, 6, 8, 86, 50, 76, 59, 61, 85, 34, 26,..."
1229,12,29,22364.284175,"[19, 56, 6, 8, 86, 50, 76, 59, 61, 85, 34, 26,..."


# pr76

## Function

In [22]:
### Get points
def get_points(data='data/pr76.txt'):
    with open(data) as f:
        line = f.readlines()
        list_data = []
        for index, data in enumerate(line):
            list_data.append(data.strip())

    points = []
    for i in range(len(list_data)):
        points.append((int(list_data[i].split(' ')[1]), int(list_data[i].split(' ')[2])))
    return points

### Get distance
def distance(point_i : int, point_j: int):
    return ((point_i[0] - point_j[0])**2 + (point_i[1] - point_j[1])**2)**(1/2)

def get_cost_matrix():
    cost_matrix = []
    for i in range(all_points):
        row = []
        for j in range(all_points):
            row.append(distance(get_points()[i], get_points()[j]))
        cost_matrix.append(row)
    return np.array(cost_matrix)


### DataFrame
def createDF(gen, ant, distance, paht):
    df = pd.DataFrame({'Gen' : gen, 'Ant' : ant,  'Distance' : distance, 'Path' : paht})
    return df

### Reset values
def reset_values():
    get_cost_matrix()

## Run

In [23]:
### Default values
all_points = len(get_points())
cost_matrix = get_cost_matrix() # Distance between i & j
cm = cost_matrix.copy()
n = np.array([[0 if i == j else 1 / cost_matrix[i][j] for i in range(all_points)] for j in range(all_points)]) # 1/distance
gens= 20
ants= 76
alpha= 3
beta= 3
p= .02

t0 = np.array([[0 if i==j else 1 for i in range(all_points)] for j in range(all_points)], dtype=float) # Default pheromone

list_gens_for_df = []
list_ants_for_df = []
list_total_distance_for_df = []
list_paths_for_df = []


list_pheromones = {}
list_shortest_distance_value = []
list_shortest_distance_index = []
list_shortest_path = []

for gen in range(gens):

    list_paths = {}
    list_total_distance = {}

    random_start = list(range(all_points))
    for ant in range(ants):
        print(f'Gen : {gen}, Ant : {ant}')
        list_gens_for_df.append(gen)
        list_ants_for_df.append(ant)

        allowed = [i for i in np.arange(all_points)]
        paths = []

        ### -------------------------------------------------- Start --------------------------------------------------
        start = random_start.pop(random.randrange(len(random_start))) # Random between 0 to 47
        selected = start

        paths.append(selected)
        # print(f'Start (Select) : {selected}')
        
        best_path = np.where(cost_matrix[paths[-1]] == sorted(cost_matrix[paths[-1]])[1])[0][0] # Return array, # sorted(cost_matrix[paths[-1]])[1] : ค่าที่ใกล้สุด
        distance_ij = sorted(cost_matrix[paths[-1]])[1] # index[0] : 0
        
        # print(f'Best paths : {best_path} ,Distance : {distance_ij}')
        # print(f'Paths : {paths}')
        
        ### Recheck best path
        # for best in enumerate(cost_matrix[paths[-1]]):
        #     print(best)

        allowed.remove(selected)
        for i in range(all_points):
            cost_matrix[selected][i] = 0
            cost_matrix[i][selected] = 0
        # print(f'Allowed paths: {allowed}')
        

        ### -------------------------------------------------- Denominator --------------------------------------------------
        list_t0 = []
        list_n = []
        for i in allowed:
            # print(f'Proromone [{selected}][{i}] : {t0[selected][i]} ** {alpha} = {t0[selected][i] ** alpha}')
            # print(f'n [{selected}][{i}] : {n[selected][i]} ** {beta} = {n[selected][i] ** beta}')
            list_t0.append(t0[selected][i] ** alpha)
            list_n.append(n[selected][i] ** beta)
        list_t0 = np.array(list_t0)
        list_n = np.array(list_n)
        denominator = sum(list_t0 * list_n)
        # print(f'Denominator : {denominator}')
        # print('-'*50)

        ### -------------------------------------------------- Probability --------------------------------------------------
        for city in range(len(allowed)):
            prob = {}
            for i in allowed:
                prob.update({i: (t0[paths[-1]][i]**alpha) * (n[paths[-1]][i]**beta) / denominator})
                # print(f'Denominator in prob : {denominator}')
            max_prob = sorted(prob.values())[-1]
            selected = list(prob.keys())[list(prob.values()).index(max_prob)]

            # for key, value in zip(prob.keys(), prob.values()):
            #     print(key, value)

            paths.append(selected)

            # print(f'Allowed : {allowed}')
            # print(f'Select : {selected}')

            ### Check walked path
            walked = 0
            for i in cost_matrix[paths[-1]]:
                if i == 0:
                    walked += 1
            # print(f'walked : {walked}')
            if walked != all_points:
                best_path = np.where(cost_matrix[paths[-1]] == sorted(cost_matrix[paths[-1]])[walked])[0][0]
                distance_ij = sorted(cost_matrix[paths[-1]])[walked]
                # print(f'Best paths : {best_path} ,Distance : {distance_ij}')

            # print(f'Paths : {paths}')

            ### Recheck best path
            # for best in enumerate(cost_matrix[paths[-1]]):
            #     print(best)

            ### Remove the path that has been walked
            allowed.remove(selected)
            for i in range(all_points):
                cost_matrix[selected][i] = 0
                cost_matrix[i][selected] = 0
            # print(f'Allowed paths: {allowed}')

            ### -------------------------------------------------- Denominator --------------------------------------------------
            list_t0 = []
            list_n = []
            for i in allowed:
                # print(f'Proromone [{selected}][{i}] : {t0[selected][i]} ** {alpha} = {t0[selected][i] ** alpha}')
                # print(f'n [{selected}][{i}] : {n[selected][i]} ** {beta} = {n[selected][i] ** beta}')
                list_t0.append(t0[selected][i] ** alpha)
                list_n.append(n[selected][i] ** beta)
            list_t0 = np.array(list_t0)
            list_n = np.array(list_n)
            denominator = sum(list_t0 * list_n)
            # print(f'Denominator : {denominator}')
            # print('-'*50)
        print(f'Paths : {paths}')
        list_paths.update({ant : paths})
        list_paths_for_df.append(paths)

        ### -------------------------------------------------- Total distance --------------------------------------------------
        total_distance = 0
        for i in range(len(paths)):
            if i != len(paths)-1:
                total_distance += (cm[paths[i]][paths[i+1]])

        print(f'Total distance : {total_distance}')
        list_total_distance.update({ant : total_distance})
        list_total_distance_for_df.append(total_distance)
    
        # print(f'List total distance : {list_total_distance}')
        # print(f'List total path : {list_paths}')

        ### -------------------------------------------------- Reset cost matrix --------------------------------------------------
        reset_values()

        # print(f'Pheromone before update : {sum(sum(t0))}')

    ### -------------------------------------------------- Update pheromone --------------------------------------------------
    shortest_distance_value = sorted(list_total_distance.values())[0]
    shortest_distance_index = list(list_total_distance.keys())[list(list_total_distance.values()).index(shortest_distance_value)]
    shortest_path = list_paths[shortest_distance_index]
    # lpwostp = list_paths.copy()
    # lpwostp.pop(shortest_distance_index) # List paths without shortest path
    for s in range(len(shortest_path)):
        for i in range(len(cm)):
            for j in range(len(cm)):
                try:
                    if i==shortest_path[s] and j==shortest_path[s+1]:
                        t_new = ((1-p)*t0[shortest_path[s]][shortest_path[s+1]]) + (100/shortest_distance_value)
                        t0[shortest_path[s]][shortest_path[s+1]] = t_new
                    else:
                        t_new = ((1-p)*t0[i][j])
                        t0[i][j] = t_new
                    # else:
                        # t0[i][j] = 0
                except:
                    pass

    # print(f'Pheromone after update : {sum(sum(t0))}')

    # list_pheromones.update({gen : t0})

    # print(f't0 of ant {gen} : {t0}')
    # print(f'List total distance : {list_total_distance}')
    # print(f'List total path : {list_paths}')
    print(f'Shortest distance value: {shortest_distance_value}')
    print(f'Shortest distance index: {shortest_distance_index}')
    print(f'Shortest path: {shortest_path}')
    list_shortest_distance_value.append(shortest_distance_value)
    list_shortest_distance_index.append(shortest_distance_index)
    list_shortest_path.append(shortest_path)
    print(f'-'*50)

print(f'List of shortest distance values : {list_shortest_distance_value}')


Gen : 0, Ant : 0
Paths : [20, 24, 23, 21, 22, 0, 1, 2, 3, 4, 19, 18, 30, 29, 28, 27, 42, 41, 53, 52, 51, 50, 65, 64, 55, 54, 57, 56, 62, 63, 61, 60, 58, 59, 40, 39, 33, 34, 35, 36, 17, 16, 10, 11, 12, 13, 14, 15, 9, 8, 5, 6, 7, 73, 37, 38, 32, 31, 25, 26, 43, 47, 46, 44, 45, 68, 67, 66, 49, 48, 70, 71, 72, 69, 75, 74]
Total distance : 139138.181286268
Gen : 0, Ant : 1
Paths : [1, 0, 22, 21, 20, 24, 23, 45, 44, 43, 47, 46, 68, 67, 66, 49, 48, 51, 52, 53, 41, 42, 27, 28, 29, 30, 18, 19, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 36, 35, 34, 33, 39, 40, 59, 58, 57, 56, 62, 63, 61, 60, 54, 55, 50, 65, 64, 70, 71, 72, 38, 37, 31, 32, 26, 25, 3, 2, 74, 75, 73, 69]
Total distance : 136263.87712897442
Gen : 0, Ant : 2
Paths : [74, 75, 0, 1, 22, 21, 20, 24, 23, 45, 44, 43, 47, 46, 68, 67, 66, 49, 48, 51, 52, 53, 41, 42, 27, 28, 29, 30, 18, 19, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 36, 35, 34, 33, 39, 40, 59, 58, 57, 56, 62, 63, 61, 60, 54, 55, 50, 65, 64, 70, 71, 72, 38, 37, 

In [24]:
list_shortest_distance_value_path = {f'{path}' : distance for path, distance in zip(list_shortest_path, list_shortest_distance_value)}
list_shortest_distance_value_path

{'[31, 32, 27, 42, 41, 53, 52, 51, 50, 65, 64, 55, 54, 57, 56, 62, 63, 61, 60, 58, 59, 40, 39, 33, 34, 35, 36, 17, 16, 10, 11, 12, 13, 14, 15, 9, 8, 5, 6, 7, 2, 3, 4, 19, 18, 30, 29, 28, 25, 26, 43, 47, 46, 44, 45, 23, 24, 20, 21, 22, 0, 1, 74, 75, 73, 37, 38, 48, 49, 66, 67, 68, 69, 70, 71, 72]': 123389.07045571336,
 '[69, 66, 67, 68, 46, 47, 43, 44, 45, 23, 24, 20, 21, 22, 0, 1, 74, 75, 2, 3, 4, 19, 18, 30, 29, 28, 25, 26, 27, 42, 41, 53, 52, 51, 50, 65, 64, 55, 54, 57, 56, 62, 63, 61, 60, 58, 59, 40, 39, 33, 34, 35, 36, 17, 16, 10, 11, 12, 13, 14, 15, 9, 8, 5, 6, 7, 73, 37, 38, 32, 31, 48, 49, 70, 71, 72]': 115185.73551405196}

## DataFrame

In [25]:
df_pr76 = createDF(list_gens_for_df, list_ants_for_df, list_total_distance_for_df, list_paths_for_df)
df_pr76

,Gen,Ant,Distance,Path
0,0,0,139138.181286,"[20, 24, 23, 21, 22, 0, 1, 2, 3, 4, 19, 18, 30..."
1,0,1,136263.877129,"[1, 0, 22, 21, 20, 24, 23, 45, 44, 43, 47, 46,..."
2,0,2,130902.546756,"[74, 75, 0, 1, 22, 21, 20, 24, 23, 45, 44, 43,..."
3,0,3,132604.291151,"[30, 18, 19, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
4,0,4,137412.642848,"[57, 56, 62, 63, 61, 60, 58, 59, 40, 39, 33, 3..."
...,...,...,...,...
1515,19,71,130149.239736,"[27, 42, 41, 53, 52, 51, 50, 65, 64, 55, 54, 5..."
1516,19,72,130758.140036,"[73, 37, 38, 32, 31, 48, 49, 70, 71, 72, 63, 6..."
1517,19,73,119230.100239,"[60, 58, 59, 40, 39, 33, 34, 35, 36, 17, 16, 1..."
1518,19,74,122629.978625,"[51, 50, 65, 64, 55, 54, 57, 56, 62, 63, 61, 6..."


## DataFrame [Shortest distace]

In [26]:
df_pr76[df_pr76.Distance == list_shortest_distance_value[-1]]

,Gen,Ant,Distance,Path
324,4,20,115185.735514,"[69, 66, 67, 68, 46, 47, 43, 44, 45, 23, 24, 2..."
423,5,43,115185.735514,"[69, 66, 67, 68, 46, 47, 43, 44, 45, 23, 24, 2..."
495,6,39,115185.735514,"[69, 66, 67, 68, 46, 47, 43, 44, 45, 23, 24, 2..."
568,7,36,115185.735514,"[69, 66, 67, 68, 46, 47, 43, 44, 45, 23, 24, 2..."
613,8,5,115185.735514,"[69, 66, 67, 68, 46, 47, 43, 44, 45, 23, 24, 2..."
716,9,32,115185.735514,"[69, 66, 67, 68, 46, 47, 43, 44, 45, 23, 24, 2..."
776,10,16,115185.735514,"[69, 66, 67, 68, 46, 47, 43, 44, 45, 23, 24, 2..."
878,11,42,115185.735514,"[69, 66, 67, 68, 46, 47, 43, 44, 45, 23, 24, 2..."
927,12,15,115185.735514,"[69, 66, 67, 68, 46, 47, 43, 44, 45, 23, 24, 2..."
1031,13,43,115185.735514,"[69, 66, 67, 68, 46, 47, 43, 44, 45, 23, 24, 2..."


In [27]:
108159

108159

# rd100

## Function

In [32]:
### Get points
def get_points(data='data/rd100.txt'):
    with open(data) as f:
        line = f.readlines()
        list_data = []
        for index, data in enumerate(line):
            list_data.append(data.strip())

    points = []
    for i in range(len(list_data)):
        points.append((int(float(list_data[i].split(' ')[1])), int(float(list_data[i].split(' ')[2]))))
    return points

### Get distance
def distance(point_i : int, point_j: int):
    return ((point_i[0] - point_j[0])**2 + (point_i[1] - point_j[1])**2)**(1/2)

def get_cost_matrix():
    cost_matrix = []
    for i in range(all_points):
        row = []
        for j in range(all_points):
            row.append(distance(get_points()[i], get_points()[j]))
        cost_matrix.append(row)
    return np.array(cost_matrix)


### DataFrame
def createDF(gen, ant, distance, paht):
    df = pd.DataFrame({'Gen' : gen, 'Ant' : ant,  'Distance' : distance, 'Path' : paht})
    return df

### Reset values
def reset_values():
    get_cost_matrix()

## Run

In [33]:
### Default values
all_points = len(get_points())
cost_matrix = get_cost_matrix() # Distance between i & j
cm = cost_matrix.copy()
n = np.array([[0 if i == j else 1 / cost_matrix[i][j] for i in range(all_points)] for j in range(all_points)]) # 1/distance
gens= 20
ants= 100
alpha= 3
beta= 3
p= .02

t0 = np.array([[0 if i==j else 1 for i in range(all_points)] for j in range(all_points)], dtype=float) # Default pheromone

list_gens_for_df = []
list_ants_for_df = []
list_total_distance_for_df = []
list_paths_for_df = []


list_pheromones = {}
list_shortest_distance_value = []
list_shortest_distance_index = []
list_shortest_path = []

for gen in range(gens):

    list_paths = {}
    list_total_distance = {}

    random_start = list(range(all_points))
    for ant in range(ants):
        print(f'Gen : {gen}, Ant : {ant}')
        list_gens_for_df.append(gen)
        list_ants_for_df.append(ant)

        allowed = [i for i in np.arange(all_points)]
        paths = []

        ### -------------------------------------------------- Start --------------------------------------------------
        start = random_start.pop(random.randrange(len(random_start))) # Random between 0 to 47
        selected = start

        paths.append(selected)
        # print(f'Start (Select) : {selected}')
        
        best_path = np.where(cost_matrix[paths[-1]] == sorted(cost_matrix[paths[-1]])[1])[0][0] # Return array, # sorted(cost_matrix[paths[-1]])[1] : ค่าที่ใกล้สุด
        distance_ij = sorted(cost_matrix[paths[-1]])[1] # index[0] : 0
        
        # print(f'Best paths : {best_path} ,Distance : {distance_ij}')
        # print(f'Paths : {paths}')
        
        ### Recheck best path
        # for best in enumerate(cost_matrix[paths[-1]]):
        #     print(best)

        allowed.remove(selected)
        for i in range(all_points):
            cost_matrix[selected][i] = 0
            cost_matrix[i][selected] = 0
        # print(f'Allowed paths: {allowed}')
        

        ### -------------------------------------------------- Denominator --------------------------------------------------
        list_t0 = []
        list_n = []
        for i in allowed:
            # print(f'Proromone [{selected}][{i}] : {t0[selected][i]} ** {alpha} = {t0[selected][i] ** alpha}')
            # print(f'n [{selected}][{i}] : {n[selected][i]} ** {beta} = {n[selected][i] ** beta}')
            list_t0.append(t0[selected][i] ** alpha)
            list_n.append(n[selected][i] ** beta)
        list_t0 = np.array(list_t0)
        list_n = np.array(list_n)
        denominator = sum(list_t0 * list_n)
        # print(f'Denominator : {denominator}')
        # print('-'*50)

        ### -------------------------------------------------- Probability --------------------------------------------------
        for city in range(len(allowed)):
            prob = {}
            for i in allowed:
                prob.update({i: (t0[paths[-1]][i]**alpha) * (n[paths[-1]][i]**beta) / denominator})
                # print(f'Denominator in prob : {denominator}')
            max_prob = sorted(prob.values())[-1]
            selected = list(prob.keys())[list(prob.values()).index(max_prob)]

            # for key, value in zip(prob.keys(), prob.values()):
            #     print(key, value)

            paths.append(selected)

            # print(f'Allowed : {allowed}')
            # print(f'Select : {selected}')

            ### Check walked path
            walked = 0
            for i in cost_matrix[paths[-1]]:
                if i == 0:
                    walked += 1
            # print(f'walked : {walked}')
            if walked != all_points:
                best_path = np.where(cost_matrix[paths[-1]] == sorted(cost_matrix[paths[-1]])[walked])[0][0]
                distance_ij = sorted(cost_matrix[paths[-1]])[walked]
                # print(f'Best paths : {best_path} ,Distance : {distance_ij}')

            # print(f'Paths : {paths}')

            ### Recheck best path
            # for best in enumerate(cost_matrix[paths[-1]]):
            #     print(best)

            ### Remove the path that has been walked
            allowed.remove(selected)
            for i in range(all_points):
                cost_matrix[selected][i] = 0
                cost_matrix[i][selected] = 0
            # print(f'Allowed paths: {allowed}')

            ### -------------------------------------------------- Denominator --------------------------------------------------
            list_t0 = []
            list_n = []
            for i in allowed:
                # print(f'Proromone [{selected}][{i}] : {t0[selected][i]} ** {alpha} = {t0[selected][i] ** alpha}')
                # print(f'n [{selected}][{i}] : {n[selected][i]} ** {beta} = {n[selected][i] ** beta}')
                list_t0.append(t0[selected][i] ** alpha)
                list_n.append(n[selected][i] ** beta)
            list_t0 = np.array(list_t0)
            list_n = np.array(list_n)
            denominator = sum(list_t0 * list_n)
            # print(f'Denominator : {denominator}')
            # print('-'*50)
        print(f'Paths : {paths}')
        list_paths.update({ant : paths})
        list_paths_for_df.append(paths)

        ### -------------------------------------------------- Total distance --------------------------------------------------
        total_distance = 0
        for i in range(len(paths)):
            if i != len(paths)-1:
                total_distance += (cm[paths[i]][paths[i+1]])

        print(f'Total distance : {total_distance}')
        list_total_distance.update({ant : total_distance})
        list_total_distance_for_df.append(total_distance)
    
        # print(f'List total distance : {list_total_distance}')
        # print(f'List total path : {list_paths}')

        ### -------------------------------------------------- Reset cost matrix --------------------------------------------------
        reset_values()

        # print(f'Pheromone before update : {sum(sum(t0))}')

    ### -------------------------------------------------- Update pheromone --------------------------------------------------
    shortest_distance_value = sorted(list_total_distance.values())[0]
    shortest_distance_index = list(list_total_distance.keys())[list(list_total_distance.values()).index(shortest_distance_value)]
    shortest_path = list_paths[shortest_distance_index]
    # lpwostp = list_paths.copy()
    # lpwostp.pop(shortest_distance_index) # List paths without shortest path
    for s in range(len(shortest_path)):
        for i in range(len(cm)):
            for j in range(len(cm)):
                try:
                    if i==shortest_path[s] and j==shortest_path[s+1]:
                        t_new = ((1-p)*t0[shortest_path[s]][shortest_path[s+1]]) + (100/shortest_distance_value)
                        t0[shortest_path[s]][shortest_path[s+1]] = t_new
                    else:
                        t_new = ((1-p)*t0[i][j])
                        t0[i][j] = t_new
                    # else:
                        # t0[i][j] = 0
                except:
                    pass

    # print(f'Pheromone after update : {sum(sum(t0))}')

    # list_pheromones.update({gen : t0})

    # print(f't0 of ant {gen} : {t0}')
    # print(f'List total distance : {list_total_distance}')
    # print(f'List total path : {list_paths}')
    print(f'Shortest distance value: {shortest_distance_value}')
    print(f'Shortest distance index: {shortest_distance_index}')
    print(f'Shortest path: {shortest_path}')
    list_shortest_distance_value.append(shortest_distance_value)
    list_shortest_distance_index.append(shortest_distance_index)
    list_shortest_path.append(shortest_path)
    print(f'-'*50)

print(f'List of shortest distance values : {list_shortest_distance_value}')


Gen : 0, Ant : 0
Paths : [70, 7, 59, 0, 17, 61, 86, 14, 62, 96, 85, 66, 12, 48, 20, 74, 81, 84, 11, 13, 3, 31, 77, 19, 73, 25, 8, 32, 9, 78, 5, 52, 98, 46, 56, 35, 63, 90, 64, 79, 80, 29, 47, 82, 50, 89, 67, 68, 2, 26, 91, 49, 45, 55, 72, 53, 37, 71, 16, 69, 18, 36, 27, 76, 92, 58, 94, 75, 65, 97, 93, 15, 44, 51, 99, 43, 28, 34, 95, 38, 54, 39, 6, 33, 40, 21, 4, 60, 10, 41, 42, 24, 23, 30, 83, 87, 57, 88, 1, 22]
Total distance : 9210.186660020665
Gen : 0, Ant : 1
Paths : [56, 35, 63, 90, 64, 79, 80, 29, 47, 82, 50, 89, 67, 70, 7, 59, 0, 17, 61, 86, 14, 62, 96, 85, 66, 12, 48, 20, 74, 81, 84, 11, 13, 3, 31, 77, 19, 73, 25, 8, 32, 9, 78, 5, 52, 98, 46, 93, 97, 65, 75, 57, 87, 83, 30, 15, 44, 51, 99, 43, 28, 34, 95, 38, 54, 39, 6, 33, 40, 21, 4, 60, 10, 41, 42, 24, 23, 68, 2, 26, 91, 49, 45, 55, 72, 53, 37, 71, 16, 69, 18, 36, 27, 76, 92, 58, 94, 88, 1, 22]
Total distance : 9268.06267122903
Gen : 0, Ant : 2
Paths : [54, 38, 95, 28, 34, 43, 99, 51, 44, 15, 97, 65, 75, 57, 87, 83, 30, 22, 1

In [34]:
list_shortest_distance_value_path = {f'{path}' : distance for path, distance in zip(list_shortest_path, list_shortest_distance_value)}
list_shortest_distance_value_path

{'[8, 25, 73, 19, 77, 31, 3, 74, 20, 48, 81, 84, 11, 13, 86, 61, 17, 0, 59, 7, 68, 14, 62, 96, 85, 66, 12, 2, 32, 9, 78, 5, 52, 98, 46, 56, 35, 63, 90, 64, 79, 80, 29, 47, 82, 50, 89, 67, 70, 4, 60, 10, 33, 6, 39, 41, 42, 24, 23, 40, 21, 54, 38, 95, 28, 34, 43, 99, 51, 44, 15, 97, 65, 75, 57, 87, 83, 30, 22, 1, 88, 58, 94, 76, 92, 27, 36, 18, 45, 49, 72, 55, 91, 26, 16, 71, 69, 37, 53, 93]': 9093.716611194877,
 '[0, 59, 7, 68, 14, 62, 96, 85, 66, 12, 48, 20, 74, 81, 84, 11, 13, 86, 61, 17, 70, 67, 82, 50, 89, 64, 79, 80, 29, 47, 90, 63, 35, 56, 46, 98, 52, 5, 78, 9, 32, 2, 3, 31, 77, 19, 73, 25, 8, 16, 71, 69, 37, 53, 72, 49, 45, 55, 91, 26, 18, 36, 27, 76, 92, 58, 94, 75, 65, 97, 93, 15, 44, 51, 99, 43, 28, 34, 95, 38, 54, 39, 6, 33, 40, 21, 4, 60, 10, 41, 42, 24, 23, 30, 83, 87, 57, 88, 1, 22]': 8738.988427915103,
 '[57, 88, 1, 22, 83, 87, 30, 65, 97, 93, 15, 44, 51, 99, 43, 28, 34, 95, 38, 54, 39, 6, 33, 40, 21, 4, 60, 10, 41, 42, 24, 23, 29, 47, 79, 80, 64, 50, 89, 82, 67, 70, 7, 5

## DataFrame

In [35]:
df_rd100 = createDF(list_gens_for_df, list_ants_for_df, list_total_distance_for_df, list_paths_for_df)
df_rd100

,Gen,Ant,Distance,Path
0,0,0,9210.186660,"[70, 7, 59, 0, 17, 61, 86, 14, 62, 96, 85, 66,..."
1,0,1,9268.062671,"[56, 35, 63, 90, 64, 79, 80, 29, 47, 82, 50, 8..."
2,0,2,9484.410638,"[54, 38, 95, 28, 34, 43, 99, 51, 44, 15, 97, 6..."
3,0,3,9430.920510,"[33, 6, 39, 41, 42, 24, 23, 40, 21, 4, 60, 10,..."
4,0,4,9122.337587,"[42, 24, 23, 41, 6, 33, 40, 21, 4, 60, 10, 35,..."
...,...,...,...,...
1995,19,95,8714.292090,"[58, 94, 75, 5, 78, 9, 32, 2, 98, 52, 46, 56, ..."
1996,19,96,9804.802355,"[17, 61, 86, 14, 62, 96, 85, 66, 12, 48, 20, 7..."
1997,19,97,9436.110164,"[98, 52, 46, 56, 35, 63, 90, 68, 7, 59, 0, 17,..."
1998,19,98,10024.156682,"[3, 31, 77, 19, 73, 25, 8, 16, 71, 69, 37, 53,..."


## DataFrame [Shortest distace]

In [36]:
df_rd100[df_rd100.Distance == list_shortest_distance_value[-1]]

,Gen,Ant,Distance,Path
318,3,18,8628.264068,"[57, 88, 1, 22, 83, 87, 30, 65, 97, 93, 15, 44..."
463,4,63,8628.264068,"[57, 88, 1, 22, 83, 87, 30, 65, 97, 93, 15, 44..."
527,5,27,8628.264068,"[57, 88, 1, 22, 83, 87, 30, 65, 97, 93, 15, 44..."
697,6,97,8628.264068,"[57, 88, 1, 22, 83, 87, 30, 65, 97, 93, 15, 44..."
704,7,4,8628.264068,"[57, 88, 1, 22, 83, 87, 30, 65, 97, 93, 15, 44..."
822,8,22,8628.264068,"[57, 88, 1, 22, 83, 87, 30, 65, 97, 93, 15, 44..."
945,9,45,8628.264068,"[57, 88, 1, 22, 83, 87, 30, 65, 97, 93, 15, 44..."
1052,10,52,8628.264068,"[57, 88, 1, 22, 83, 87, 30, 65, 97, 93, 15, 44..."
1135,11,35,8628.264068,"[57, 88, 1, 22, 83, 87, 30, 65, 97, 93, 15, 44..."
1240,12,40,8628.264068,"[57, 88, 1, 22, 83, 87, 30, 65, 97, 93, 15, 44..."


In [ ]:
7910